In [1]:
import csv
import numpy as np
import pandas as pd

file_name = '../data/artwork_transactions_sample.csv'
artwork_transactions = pd.read_csv(file_name)


In [14]:
artwork_transactions.head()

,index,id,trans_type,display_price,calc_price,eval_price,eval_price_min,eval_price_max,trans_category,trans_art_type,...,info_update_time,trade_channel_id,origin_url,created_at,updated_at,transaction_date,lot,auction_id,company_id,artist
0,0,1310631,auction,流拍,NaN,"USD 1,600-2,400",NaN,NaN,综合媒材,西画雕塑,...,2015-07-12 05:07:34,4058,http://auction.artron.net/paimai-art0029920063/,2014-01-18 00:00:00,2014-01-18 00:00:00,2014-01-17 16:00:00,0063,1331,177,1621
1,1,1476875,auction,流拍,NaN,"RMB 80,000-120,000",NaN,NaN,绘画,中国书画,...,2015-07-12 13:30:32,2415,http://auction.artron.net/paimai-art5017610140/,2012-05-04 00:00:00,2012-05-04 00:00:00,2012-05-03 16:00:00,0140,2344,3,47595
2,2,1698094,auction,流拍,NaN,"HKD 3,000-5,000",NaN,NaN,绘画,中国书画,...,2015-07-12 20:07:52,2985,http://auction.artron.net/paimai-art64950475/,2010-05-27 00:00:00,2010-05-27 00:00:00,2010-05-26 16:00:00,0475,3310,101,8370
3,3,1085516,auction,"RMB 22,600",22600.0,"RMB 20,000-30,000",NaN,NaN,书法,中国书画,...,2015-07-11 14:19:01,13270,http://auction.artron.net/paimai-art5036120262/,2013-07-09 00:00:00,2013-07-09 00:00:00,2013-07-08 16:00:00,0262,1824,316,20339
4,4,730475,auction,流拍,NaN,"RMB 22,000-32,000",NaN,NaN,书法,中国书画,...,2015-07-10 09:14:31,8413,http://auction.artron.net/paimai-art5027780224/,2012-12-16 00:00:00,2012-12-16 00:00:00,2012-12-15 16:00:00,0224,1219,238,94447


In [3]:
artist_price = pd.read_csv('../data/artist_price.csv')

In [4]:
artist_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312869 entries, 0 to 2312868
Data columns (total 3 columns):
id            int64
calc_price    float64
artist_id     int64
dtypes: float64(1), int64(2)
memory usage: 52.9 MB


In [5]:
artist_price['calc_price'] = artist_price['calc_price'].fillna(0)
artist_price = artist_price[artist_price.calc_price != 0]
artist_price = artist_price.sort_values(by='artist_id')

In [6]:
def find_out_lables(column):
    lables = []
    for value in column:
        if value not in lables:
            lables.append(value)
    #print(lables)
    return lables

artists = find_out_lables(artist_price['artist_id'])

In [7]:
len(artists)

69611

In [8]:
artist_price_dic = artist_price.groupby('artist_id')['calc_price'].mean()


In [9]:
artist_numtran_dic = artist_price.groupby('artist_id')['calc_price'].count()

In [23]:
artist_numtran_dic[1621]

32

In [10]:
artist_numtran_dic

artist_id
1          64
2           1
8           1
9           1
10          9
11          1
12          1
15        338
17          1
18         30
22          1
24          1
25          1
26          1
27          1
31          1
33          1
34          1
35          1
36        199
37          3
38          2
40          3
41          2
42          1
45          1
47          1
48          1
49        134
50          2
         ... 
123353      1
123354      1
123355    895
123356     10
123358     51
123361      2
123362      5
123365      1
123376      1
123378      1
123380      1
123381      1
123383      1
123384      1
123386      1
123387      2
123394      1
123396      5
123398      1
123399      1
123403      4
123405      5
123406      1
123407      1
123409      1
123410      2
123461      2
123464      1
123467      1
123470      1
Name: calc_price, Length: 69611, dtype: int64

In [11]:
#drop column with no info

artwork_transactions.drop(['auction_info_link'],axis=1,inplace=True)
artwork_transactions.drop(['collector'],axis=1,inplace=True)
artwork_transactions.drop(['sell_agency'],axis=1,inplace=True)
artwork_transactions.drop(['image_url'],axis=1,inplace=True)
artwork_transactions.drop(['icon_img_url'],axis=1,inplace=True)
artwork_transactions.drop(['search_string'],axis=1,inplace=True)

artwork_transactions.drop(['eval_price_curr'],axis=1,inplace=True)        #information in eval_price
artwork_transactions.drop(['trade_curr'],axis=1,inplace=True)             #only one value
artwork_transactions.drop(['comments_count'],axis=1,inplace=True) 
artwork_transactions.drop(['likes_count'],axis=1,inplace=True) 


#删除‘未提供’价格的 row

artwork_transactions = artwork_transactions[artwork_transactions.display_price != '未提供']
artwork_transactions = artwork_transactions.reset_index()

In [12]:
artwork_transactions['trans_category']=artwork_transactions['trans_category'].fillna('unclear')


        
#make new column artist

artist_artworktransaction_dic = pd.read_csv('../data/artist_artworktransaction.csv')
artwork_transactions['artist'] = range(len(artwork_transactions))
for i in range(len(artwork_transactions)):
    artwork_transactions['artist'][i] = artist_artworktransaction_dic.artist_id[artist_artworktransaction_dic.artwork_transaction_id == artwork_transactions.id[i]]
        

/Users/eva/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [25]:
# 作者的平均价格
artwork_transactions['artist_price'] = [0] * len(artwork_transactions)
for i in range(len(artwork_transactions)):
    if type(artwork_transactions.artist[i]) == int:
        artwork_transactions['artist_price'][i] = artist_price_dic[artwork_transactions.artist[i]]
        

# 作者的流通次数
artwork_transactions['artist_numtran'] = [0] * len(artwork_transactions)
for i in range(len(artwork_transactions)):
    if type(artwork_transactions.artist[i]) == int:
        artwork_transactions['artist_numtran'][i] = artist_numtran_dic[artwork_transactions.artist[i]]
        

In [ ]:
# about time form transform
import datetime
import time

def texts_to_dates(texts):
    dates = []
    for text in texts:
        date = datetime.datetime.strptime(text, '%Y-%m-%d %H:%M:%S').date()
        dates.append(date)
    return dates


def texts_to_strptime(texts):
    counter =0
    dates = np.array(range(len(texts)))
    for text in texts:
        date = datetime.datetime.strptime(text, '%Y-%m-%d %H:%M:%S').date()
        date_s = time.mktime(date.timetuple())
        #print(dates[counter])
        dates[counter] = date_s
        #print (dates[:10])
        counter +=1
    #print(dates)
    return dates

def get_age_days(date1s,date2s):
    age_days=[]
    for i in range(len(date1s)):
        age_days.append((date1s[i] - date2s[i]).days)
    return age_days


artwork_transactions['transaction_date'] = texts_to_strptime(artwork_transactions.transaction_date)
artwork_transactions['transaction_date'] = (artwork_transactions['transaction_date'])

In [26]:
import jieba

a = '开发者可以指定自己自定义的词典，以便包含 jieba 词库里没有的词。虽然 jieba 有新词识别能力，但是自行添加新词可以保证更高的正确率'
b = jieba.cut(a)

In [27]:
type(b)

generator

In [28]:
print(b)

<generator object Tokenizer.cut at 0x117b71c50>


In [30]:
c = ' '.join(b)

In [31]:
type (c)

str